In [1]:
!pip install -q langchain faiss-cpu sentence-transformers groq python-dotenv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.6 MB/s eta 0:00:00


In [2]:
import os
os.environ["GROQ_API_KEY"] = "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"


In [3]:
from google.colab import files
uploaded = files.upload()


Saving 9241544228_eng.pdf to 9241544228_eng.pdf


In [5]:
!pip install -q pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 76.1 MB/s eta 0:00:00


In [6]:
import fitz  # PyMuPDF

# Get filename
filename = list(uploaded.keys())[0]

# Extract text from all pages
doc = fitz.open(filename)
text = ""
for page in doc:
    text += page.get_text()

print(f"✅ Extracted {len(text)} characters from PDF.")


✅ Extracted 660457 characters from PDF.


In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Use RecursiveCharacterTextSplitter to chunk the raw text
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,     # Max size per chunk (in characters)
    chunk_overlap=50,   # Overlap to retain context
    separators=["\n\n", "\n", ".", " ", ""]
)

# Split into documents
documents = text_splitter.create_documents([text])

# Preview how many chunks created
print(f"✅ Total chunks created: {len(documents)}")
print("\n🔹 Sample chunk:\n")
print(documents[0].page_content[:1000])


✅ Total chunks created: 1438

🔹 Sample chunk:

ThelCD-10
Classification
of Mental and
Behavioural
Disorders
Clinical
descriptions
and diagnostic
guidelines
| 
World Health Organization
I 
Geneva
I 
1992
ICD-10
Reprinted 1993, 1994, 1995, 1998, 2000, 2002, 2004
WHO Library Cataloguing in Publication Data
The ICD-10 classification of mental and behavioural
disorders : clinical descriptions and diagnostic guidelines.
1.Mental disorders — classification 
2.Mental disorders — diagnosis
ISBN 92 4 154422 8 
(NLM Classification: WM 15)


In [8]:
!pip install -q sentence-transformers faiss-cpu langchain


In [10]:
!pip install -U langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00


In [11]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# Load the embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create FAISS vector store from documents
vectorstore = FAISS.from_documents(documents, embedding_model)

# Save FAISS index (optional, for later reuse)
vectorstore.save_local("faiss_icd10_index")

print("✅ FAISS vector store created with embedded document chunks.")


/tmp/ipython-input-11-2234839673.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public mo

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ FAISS vector store created with embedded document chunks.


In [12]:
!pip install groq


In [13]:
from groq import Groq

groq_api_key = "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"
client = Groq(api_key=groq_api_key)


In [16]:
def rag_qa(query, k=5):
    # 1. Retrieve top-k relevant chunks from FAISS
    docs = vectorstore.similarity_search(query, k=k)
    context = "\n\n".join([doc.page_content for doc in docs])

    # 2. Build the prompt
    final_prompt = f"""You are a helpful medical assistant.
Use the following context from the ICD-10 to answer the question clearly.

Context:
{context}

Question: {query}

Answer:"""

    # 3. Query Groq LLM
    response = client.chat.completions.create(
        model="llama3-8b-8192",  # ✅ Use supported model
        messages=[{"role": "user", "content": final_prompt}],
        temperature=0.2,
        max_tokens=512
    )

    return response.choices[0].message.content.strip()


In [17]:
response = rag_qa("What are the diagnostic criteria for schizophrenia?")
print("🔹Answer:\n", response)


🔹Answer:
 According to the provided context from the ICD-10, the diagnostic criteria for schizophrenia are not explicitly stated. However, it is mentioned that the general criteria for a diagnosis of schizophrenia must be satisfied, and the introduction to F20 provides more information on the diagnostic criteria for schizophrenia.

However, if we look at the diagnostic guidelines for other types of schizophrenia, such as hebephrenia and simple schizophrenia, we can infer some of the diagnostic criteria for schizophrenia. For example, the guidelines for simple schizophrenia state that the individual must have a history of at least one clear-cut psychotic episode meeting the diagnostic criteria for schizophrenia, and the intensity and frequency of florid symptoms such as delusions and hallucinations must have been minimal or substantially reduced for a period of at least 1 year.

It's also mentioned that the disorder runs a chronic course with fluctuations of intensity, and occasionally 

In [18]:
response = rag_qa("Give me the correct coded classification for the following diagnosis: Recurrent depressive disorder, currently in remission")
print("🔹Answer:\n", response)


🔹Answer:
 Based on the provided context from the ICD-10, the correct coded classification for the diagnosis "Recurrent depressive disorder, currently in remission" is:

F33.4

This code is specified for "Recurrent depressive disorder, currently in remission", which indicates that the individual has had previous episodes of depression, but is currently not experiencing any symptoms.
